In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import countDistinct, col

In [2]:
spark = SparkSession.builder.appName("user_engagement").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/27 22:05:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_users = spark.read.format('csv').options(header='true', inferSchema='true').load("takehome_users.csv")
df_engagement = spark.read.format('csv').options(header='true', inferSchema='true').load("takehome_user_engagement.csv")

In [4]:
df_users.createOrReplaceTempView("users")
df_engagement.createOrReplaceTempView("engagement")

In [5]:
df_diff_days = spark.sql("""
    SELECT user_id, 
           time_stamp, 
           datediff(
               lead(time_stamp, 1) OVER(PARTITION BY user_id ORDER BY time_stamp), 
               time_stamp
           ) as diff_days 
    FROM engagement
""")

In [6]:
df_diff_days.createOrReplaceTempView("diff_days")


In [7]:
df_adopted_users = spark.sql("""
    SELECT user_id 
    FROM (
        SELECT user_id,
               sum(diff_days) OVER (
                   PARTITION BY user_id 
                   ORDER BY time_stamp 
                   ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING
               ) as sum_days
        FROM diff_days
    )
    WHERE sum_days <= 7
""")

In [8]:
df_diff_days.show()

+-------+-------------------+---------+
|user_id|         time_stamp|diff_days|
+-------+-------------------+---------+
|      1|2014-04-22 03:53:30|     null|
|      2|2013-11-15 03:45:04|       14|
|      2|2013-11-29 03:45:04|       10|
|      2|2013-12-09 03:45:04|       16|
|      2|2013-12-25 03:45:04|        6|
|      2|2013-12-31 03:45:04|        8|
|      2|2014-01-08 03:45:04|       26|
|      2|2014-02-03 03:45:04|        5|
|      2|2014-02-08 03:45:04|        1|
|      2|2014-02-09 03:45:04|        4|
|      2|2014-02-13 03:45:04|        3|
|      2|2014-02-16 03:45:04|       21|
|      2|2014-03-09 03:45:04|        4|
|      2|2014-03-13 03:45:04|       18|
|      2|2014-03-31 03:45:04|     null|
|      3|2013-03-19 23:14:52|     null|
|      4|2013-05-22 08:09:28|     null|
|      5|2013-01-22 10:14:20|     null|
|      6|2013-12-19 03:37:06|     null|
|      7|2012-12-20 13:24:32|     null|
+-------+-------------------+---------+
only showing top 20 rows



In [9]:
df_adopted_users.createOrReplaceTempView("adopted_users")

df_users = spark.sql("""
    SELECT users.*,
           CASE WHEN adopted_users.user_id IS NOT NULL THEN TRUE ELSE FALSE END as is_adopted
    FROM users
    LEFT JOIN adopted_users ON users.object_id = adopted_users.user_id
""")

In [14]:
df_users.printSchema()

root
 |-- object_id: integer (nullable = true)
 |-- creation_time: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- creation_source: string (nullable = true)
 |-- last_session_creation_time: integer (nullable = true)
 |-- opted_in_to_mailing_list: integer (nullable = true)
 |-- enabled_for_marketing_drip: integer (nullable = true)
 |-- org_id: integer (nullable = true)
 |-- invited_by_user_id: integer (nullable = true)
 |-- is_adopted: boolean (nullable = false)



In [15]:
df_users.select("is_adopted").show()

+----------+
|is_adopted|
+----------+
|     false|
|     false|
|     false|
|     false|
|     false|
|     false|
|     false|
|     false|
|     false|
|      true|
|      true|
|      true|
|      true|
|      true|
|      true|
|      true|
|      true|
|      true|
|      true|
|      true|
+----------+
only showing top 20 rows



In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [22]:
df_user_dummies = pd.get_dummies(df_users, columns=['creation_source'])

In [25]:
print(list(df_user_dummies.columns))

[DataFrame[object_id: int, creation_time: timestamp, name: string, email: string, creation_source: string, last_session_creation_time: int, opted_in_to_mailing_list: int, enabled_for_marketing_drip: int, org_id: int, invited_by_user_id: int, is_adopted: boolean]]


In [29]:
X = df_user_dummies.drop("is_adopted", axis=1)
y = df_user_dummies["is_adopted"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

KeyError: "['is_adopted'] not found in axis"

In [31]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


string_indexer = StringIndexer(inputCol="creation_source", outputCol="creation_source_index")
encoder = OneHotEncoder(inputCol="creation_source_index", outputCol="creation_source_encoded")
assembler = VectorAssembler(inputCols=["object_id", "creation_source_encoded", "last_session_creation_time",
                                       "opted_in_to_mailing_list", "enabled_for_marketing_drip", "org_id",
                                       "invited_by_user_id"], outputCol="features")


selected_features = ["features", "is_adopted"]
df_selected_features = df_features.select(selected_features)



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `features` cannot be resolved. Did you mean one of the following? [`is_adopted`, `users`.`name`, `users`.`email`, `users`.`org_id`, `users`.`object_id`].;
'Project ['features, is_adopted#83]
+- Project [object_id#17, creation_time#18, name#19, email#20, creation_source#21, last_session_creation_time#22, opted_in_to_mailing_list#23, enabled_for_marketing_drip#24, org_id#25, invited_by_user_id#26, is_adopted#83, creation_source_index#219, UDF(cast(creation_source_index#219 as double), 0) AS creation_source_encoded#239]
   +- Project [object_id#17, creation_time#18, name#19, email#20, creation_source#21, last_session_creation_time#22, opted_in_to_mailing_list#23, enabled_for_marketing_drip#24, org_id#25, invited_by_user_id#26, is_adopted#83, UDF(cast(creation_source#21 as string)) AS creation_source_index#219]
      +- Project [object_id#17, creation_time#18, name#19, email#20, creation_source#

The main goal was to to create a view that had an is_adopted column and this would be the main feature I would be looking at. As you can see, this failed, and this is because for some reason although the column is being created Pandas data frame is not reading the column. Nor is the PySpark dataframe machine learning tools. My query building was thinking that I could find the difference in the dates from engagement, and use that to create a view to query even further.